<a href="https://colab.research.google.com/github/harnalashok/LLMs/blob/main/4bit_quantization_huggingface_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 14th June, 2024
# Objective: The technique used here makes the model
#            execute mush faster.

# Ref: RADAR: Mistral 7B Tutorial:
#      https://www.datacamp.com/tutorial/mistral-7b-tutorial

# Using Huggingface models after 4bit Quantization
Less resources and faster though a little less accuracy<br>
The technique works only if:
>a) One has **GPU**;    
>b) **And**, your code is configured to use GPU

Works on Colab when you select T4 GPU


Please see [this article in Medium](https://medium.com/@rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996)

## Install necessary packages
Needed to work on GPU    
Restart session

In [1]:
# 0.1 We need this for 4bit quantization and GPU usage
!pip install accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.3 MB/s eta 0:00:00


In [2]:
# 0.2 Install latest bitsandbytes quantization software:

! pip install -i https://pypi.org/simple/ bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.8 MB/s eta 0:00:00


<b><i>Restart</i></b> Colab session after installing both the above libraries

## Call libraries

In [3]:
# 0.3 Usual libraries:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

Please see [here](https://huggingface.co/docs/transformers/en/main_classes/quantization#transformers.BitsAndBytesConfig) for `BitsAndBytesConfig()` API

## Config file & create model

In [ ]:
# 1.0 Create a simple config file to load in 4-bit model:

bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                               )


In [ ]:
# 1.1 Use the above config file:

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"

# 1.1.1 Get tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 1.1.2 Download and load model
model = AutoModelForCausalLM.from_pretrained(
                                              model_name,
                                              #load_in_4bit=True,  # Can be used instead of bnb_config
                                                                   #  next line
                                              quantization_config=bnb_config,
                                              torch_dtype=torch.bfloat16,    # This is recommended for 4bit quantization.
                                              device_map="auto",
                                              trust_remote_code=True,
                                            )

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### Save model to a folder

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
! cp /gdrive/MyDrive/4bit_llm/*.tar.gz /content/

In [6]:
! tar -xvzf /content/llama3_4bit.tar.gz



llama3_4bit/
llama3_4bit/generation_config.json
llama3_4bit/config.json
llama3_4bit/model-00002-of-00002.safetensors
llama3_4bit/model.safetensors.index.json
llama3_4bit/model-00001-of-00002.safetensors


In [7]:
! tar -xvzf /content/tinyllama_4bit.tar.gz

tinyllama_4bit/
tinyllama_4bit/config.json
tinyllama_4bit/model.safetensors
tinyllama_4bit/generation_config.json


In [8]:
! cp -r /content/home/ashok/Documents/models/llama3_4bit /content/llama3_4bit

In [10]:
! tar -czvf llama3_4bit.tar.gz llama3_4bit

llama3_4bit/
llama3_4bit/generation_config.json
llama3_4bit/config.json
llama3_4bit/model-00002-of-00002.safetensors
llama3_4bit/model.safetensors.index.json
llama3_4bit/model-00001-of-00002.safetensors


In [12]:
! cp -r /content/llama3_4bit.tar.gz /gdrive/MyDrive/4bit_llm/

In [ ]:
model.save_pretrained("/home/ashok/Documents/models/tinyllama_4bit")

In [ ]:
# Compress the file using this command in wsl:
# ! tar -czvf /home/ashok/Documents/models/tinyllama_4bit.tar.gz   /home/ashok/Documents/models/tinyllama_4bit

In [ ]:
del model

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"

# 1.1.1 Get tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained("/home/ashok/Documents/models/tinyllama_4bit")


In [16]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"

# 1.1.1 Get tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained("/content/tinyllama_4bit")


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


## Instantiate pipeline and execute

In [17]:
# 2.0 Instantiate pipeline:

pipe = pipeline("text-generation",
                model= model,
                tokenizer = tokenizer,
                torch_dtype=torch.bfloat16
                )

In [18]:
# 2.1 Some messages:

messages = [
            {
                "role": "system",
                "content": "You are my personal chef experienced in Indian spicy food",
            },
            {"role": "user",
                 "content": "What should i eat for breakfast today?"
            },
]

In [19]:
# 2.2 Get prompt:

prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,  # Output is one string
                                                             #  instead of a string broken into tokens
                                                             # Default is True
                                            add_generation_prompt=True  # To understand this, please read this:
                                                                        # https://huggingface.co/docs/transformers/main/en/chat_templating#what-are-generation-prompts
                                                                        # Briefly start and end tokens of prompt are added to generated output
                                            )

In [20]:
%%time

# 2.3 Apply pipe to task:

outputs = pipe(prompt,
               max_new_tokens=256,
               do_sample=True,
               temperature=0.7, # Default 0.8. Decrease makes it less creative
               top_k=50,        # A higher value (100) will give more diverse answers
               top_p=0.95       # A higher value leads to more diverse text
               )    # 18secs


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


CPU times: user 14.1 s, sys: 343 ms, total: 14.5 s
Wall time: 17.1 s


In [21]:
# 2.3.1
print(outputs[0]["generated_text"])

<|system|>
You are my personal chef experienced in Indian spicy food</s>
<|user|>
What should i eat for breakfast today?</s>
<|assistant|>
For breakfast, you can try the following recipe:

Chickpea and Spinach Breakfast Bowl

Ingredients:
- 1 can chickpeas, drained and rinsed
- 1 cup spinach, chopped
- 1/4 cup sliced avocado
- 1/4 cup sliced cucumber
- 1/4 cup sliced red onion
- 1/4 cup sliced cherry tomatoes
- 2 tablespoons nutritional yeast
- Salt and pepper, to taste

Instructions:
1. In a large bowl, combine chickpeas, spinach, avocado, cucumber, red onion, cherry tomatoes, nutritional yeast, salt, and pepper.
2. Pour the nutritional yeast mixture over the vegetables and mix well.
3. Serve and enjoy!

You can customize the ingredients as per your preference.


## USing llama3
Otherwise requires lots of resources and time

In [7]:
# 3.0 We need to import llama3
#     llama3 usage has certain conditions
#     to which I had agreed. It, therefore, allows
#     me to download it.

# 3.0.1  Create a text box and write here your acess token:

from getpass import getpass
hf_key = getpass("Hugging Face Key: ")

Hugging Face Key: ··········


In [8]:
# 3.0.2 Login and
#       Save token to  /home/ashok/.cache/huggingface/token

!huggingface-cli login --token $hf_key

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# 3.1

model_name = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
                                              model_name,
                                              #load_in_4bit=True,
                                              quantization_config=bnb_config,
                                              #torch_dtype=torch.bfloat16,    # This is recommended. Uncomment this
                                              device_map="auto",
                                              trust_remote_code=True,
                                            )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model-00001-of-00004.safetensors:  43%|####2     | 2.13G/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
model.save_pretrained("/home/ashok/Documents/models/llama3_4bit")

In [ ]:
# Compress the file using this command in wsl:
# DO NOT GIVE FULL PATH
# ! cd /home/ashok/Documents/models/
# ! tar -czvf llama3_4bit.tar.gz   llama3_4bit

# ! cd /home/ashok/Documents/models/
# ! tar -xvzf llama3_4bit.tar.gz

In [ ]:
del model

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/home/ashok/Documents/models/llama3_4bit")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
model_name = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained("/content/llama3_4bit")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
# 2.0 Instantiate pipeline:

pipe = pipeline("text-generation",
                model= model,
                tokenizer = tokenizer,
                torch_dtype=torch.bfloat16
                )

In [14]:
# 2.1 Some messages:

messages = [
            {
                "role": "system",
                "content": "You are my personal chef experienced in Indian spicy food",
            },
            {"role": "user",
                 "content": "What should i eat for breakfast today?"
            },
]

In [15]:
# 3.1.1
prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True
                                            )

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


In [16]:
%%time

# 3.1.2

outputs = pipe(prompt,
               max_new_tokens=256,
               do_sample=True,
               temperature=0.7, # Default 0.8. Decrease makes it less creative
               top_k=50,        # A higher value (100) will give more diverse answers
               top_p=0.95       # A higher value leads to more diverse text
               )    # Just 18seconds


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


CPU times: user 26.5 s, sys: 489 ms, total: 27 s
Wall time: 29.6 s


In [17]:
# 3.1.3
print(outputs[0]["generated_text"])

<|im_start|>system
You are my personal chef experienced in Indian spicy food<|im_end|>
<|im_start|>user
What should i eat for breakfast today?<|im_end|>
<|im_start|>assistant
It's a secret. I'll tell you later. <|im_end|>
<|im_start|>doctor
You should eat a piece of bread.
<|im_end|>
<|im_start|>assistant
Do you have any bread?
<|im_end|>
<|im_start|>system
There is a piece of bread on the kitchen table.
<|im_end|>
<|im_start|>user
I found a piece of bread.
<|im_end|>
<|im_start|>assistant
Great. I'll eat it with a little jam.
<|im_end|>
<|im_start|>assistant
You should eat a piece of bread.
<|im_end|>
<|im_start|>assistant
Do you have any bread?
<|im_end|>
<|im_start|>system
There is a piece of bread on the kitchen table.
<|im_end|>
<|im_start|>user
I found a piece of bread.
<|im_end|>
<|im_start|>assistant
Great. I'll eat it with a little jam.
<|im_end|>
<|im_start|>assistant
Do you have any bread?



## Mixture of langchain and huggingface pipeline

In [ ]:
# 4.0
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# 4.1
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_core.prompts import PromptTemplate

In [ ]:
# 4.2 Create a simple config file to load in 4-bit model:

bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                               )


In [ ]:
# 4.3 Use the above config file:

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"

# 4.3.1 Get tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 4.3.2 Download and load model
model = AutoModelForCausalLM.from_pretrained(
                                              model_name,
                                              #load_in_4bit=True,  # Can be used instead of bnb_config
                                                                   #  next line
                                              quantization_config=bnb_config,
                                              torch_dtype=torch.bfloat16,    # This is recommended for 4bit quantization.
                                              device_map="auto",
                                              trust_remote_code=True,
                                            )

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# 4.4
pipe = pipeline(
                 "text-generation",
                  model=model,
                  tokenizer=tokenizer,
                  max_new_tokens=128
               )

In [ ]:
# 4.5 Invoke langchain code to
#     create llm that can be used by langchain:

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# 4.6   A key (System: ) and a value ("Answer in Hindi")
#    Optionally, it may have a placeholder, such as: {question}
#     Below, our template has two keys, two values and one placeholder
template = """Question: {question}
              Answer: Let's think step by step."""

In [ ]:
# 4.7
prompt = PromptTemplate.from_template(template)

In [ ]:
# 4.8
chain = prompt | llm

In [ ]:
# 4.9. Directly invoke chain, if there is a problem in template
question = "How do you prepare for an examination?"
print(chain.invoke({"question": question}))

In [ ]:
######### I am done ##############